In [16]:

# Assuming that the `train_model` function is defined in a file named `training.py`
from training import train_model

# Call the function with required parameters
train_model(
    data_path="./objaverse_processed/processed_models",
    num_epochs=2,
    batch_size=10,
    learning_rate=1e-4,
    model_save_path="best_model.pth"
)


Loading data from ./objaverse_processed/processed_models...
Found 1000 files in ./objaverse_processed/processed_models


100%|██████████| 80/80 [03:51<00:00,  2.90s/it]


Epoch [1/2], Loss: 0.9572
Epoch [1/2], Test Loss: 0.7044


100%|██████████| 80/80 [03:39<00:00,  2.75s/it]


Epoch [2/2], Loss: 0.6537
Epoch [2/2], Test Loss: 0.6580
Training complete. Best test loss: 0.6580


0.6579547703266144

In [14]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
